# SQL Reference

## General SQL
[Docs](https://docs.blazingdb.com/docs/queries)

### CASE
[Docs](https://docs.blazingdb.com/docs/queries#case)

```sql
SELECT CASE WHEN column_A > 5 THEN 1 ELSE 0 END FROM table_A
```

In [9]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

BlazingContext ready


#### CASE -- Single Filter

In [ ]:
query = '''
        select 
            CASE 
                WHEN fare_amount > 20 THEN 1 
                ELSE 0 END
        from 
            taxi
            '''
bc.sql(query)

#### CASE -- Multiple Filters

In [ ]:
query = '''
        select 
            CASE 
                WHEN fare_amount > 20 THEN 1 
                ELSE 0 END, 
            CASE 
                WHEN tpep_pickup_datetime like '%-01-01%' THEN 1 
                ELSE 0 END
        from
            taxi
            '''
bc.sql(query)

### CAST
[Docs](https://docs.blazingdb.com/docs/queries#cast)

```sql
SELECT CAST(column_A AS type) FROM table_A
```

In [10]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

BlazingContext ready


#### CAST -- Single Filter

In [ ]:
query = '''
        select 
            CAST(fare_amount AS int) 
        from 
            taxi
            '''
bc.sql(query)

#### CAST -- Multiple Filters

In [ ]:
query = '''
        select 
            CAST(passenger_count AS float) passenger_count_float, 
            CAST(fare_amount AS int) fare_amount_int
        from
            taxi
            '''
bc.sql(query)

#### CAST after WHERE

In [ ]:
query = '''
        select
            *
        from
            taxi
            WHERE
                DAYOFMONTH(CAST(tpep_pickup_datetime || '.0' AS TIMESTAMP)) <= 7
                '''
bc.sql(query)

### LIKE
[Docs](https://docs.blazingdb.com/docs/queries#like)

```sql
SELECT * FROM table_A WHERE column_A LIKE '%foo%'
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### LIKE -- Single Filter

In [ ]:
bc.sql("select * from taxi WHERE tpep_pickup_datetime LIKE '%15-01%'")

#### LIKE -- Multiple Filters (AND)

In [ ]:
query = '''
        select    
            * 
        from 
            taxi 
            WHERE 
                tpep_pickup_datetime LIKE '%01-01%' 
                AND tpep_dropoff_datetime LIKE '%01-01%'
                '''
bc.sql(query)

#### LIKE -- Multiple Filters (OR)

In [ ]:
query = '''
        select 
            * 
        from 
            taxi 
        WHERE 
            tpep_pickup_datetime LIKE '%01-01%' 
            OR tpep_dropoff_datetime LIKE '%01-01%'
            '''
bc.sql(query)

#### LIKE -- Multiple Filters (AND/OR)

In [ ]:
query = '''
        select 
            * 
        from 
            taxi 
        WHERE 
            (tpep_pickup_datetime LIKE '%01-01%' OR tpep_dropoff_datetime LIKE '%01-01%') 
            AND dropoff_x LIKE '-822%'
            '''
bc.sql(query)

### GROUP BY
[Docs](https://docs.blazingdb.com/docs/queries#group-by)

Groups result set of a query by one or more columns.

```sql
SELECT * FROM table_a GROUP BY column_a
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

In [ ]:
bc.sql('select passenger_count from taxi GROUP BY passenger_count')

In [ ]:
query = '''
        select
            passenger_count,
            VendorID
        from
            taxi
        GROUP BY
            passenger_count, 
            VendorID
            '''
bc.sql(query)

#### SUM

In [ ]:
# --SUM without GROUP BY
bc.sql('select SUM(fare_amount) from taxi')

In [ ]:
# --SUM with GROUP BY
bc.sql('select SUM(fare_amount) from taxi GROUP BY passenger_count')

#### AVG

In [ ]:
# --AVG without GROUP BY
bc.sql('select AVG(fare_amount) from taxi')

In [ ]:
# --AVG with GROUP BY
bc.sql('select AVG(fare_amount) from taxi GROUP BY passenger_count')

#### MIN

In [ ]:
# --MIN without GROUP BY
bc.sql('select MIN(fare_amount) FROM taxi')

In [ ]:
# --MIN with GROUP BY
bc.sql('select MIN(fare_amount) from taxi GROUP BY passenger_count')

#### MAX

In [ ]:
# --MAX without GROUP BY
bc.sql('select MAX(fare_amount) FROM taxi')

In [ ]:
# --MAX with GROUP BY
bc.sql('select MAX(fare_amount) from taxi GROUP BY passenger_count')

#### COUNT

In [ ]:
# --COUNT without GROUP BY
bc.sql('select COUNT(fare_amount) from taxi')

In [ ]:
# --COUNT with GROUP BY
bc.sql('select COUNT(fare_amount) FROM taxi GROUP BY passenger_count')

#### DISTINCT

In [ ]:
# --DISTINCT without GROUP BY
bc.sql('select DISTINCT(passenger_count) from taxi')

In [ ]:
# --DISTINCT with GROUP BY
bc.sql('select DISTINCT(passenger_count) from taxi GROUP BY passenger_count')

### JOIN
[Docs](https://docs.blazingdb.com/docs/queries#join)

Combine rows from two or more tables, based on a related column or columns between them.

#### INNER JOIN
[Docs](https://docs.blazingdb.com/docs/queries#inner)

```sql
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
INNER JOIN table_B 
ON (table_A.column_A = table_B.column_B)
```

In [5]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi_a', '../../../data/sample_taxi.parquet')
bc.create_table('taxi_b', '../../../data/sample_taxi.csv', header=0)

BlazingContext ready


#### INNER JOIN SINGLE COLUMN

In [2]:
query = '''
        select 
            A.tpep_pickup_datetime, A.VendorID,
            B.passenger_count, B.trip_distance, B.fare_amount
        from 
            taxi_a as A
        INNER JOIN 
            taxi_b as B
            ON A.tpep_pickup_datetime = B.tpep_pickup_datetime
            '''
bc.sql(query)

Error found
fail: 
 org.apache.calcite.runtime.CalciteContextException: From line 9, column 18 to line 9, column 37: Column 'tpep_pickup_datetime' not found in table 'A'
Parsing Error


#### INNER JOIN MULTI-COLUMN

In [3]:
query = '''
        select 
            A.tpep_pickup_datetime, A.VendorID,
            B.passenger_count, B.trip_distance, B.fare_amount
        from 
            taxi_a as A
        INNER JOIN 
            taxi_b as B
            ON A.tpep_pickup_datetime = B.tpep_pickup_datetime
            AND A.VendorID = B.VendorID
            '''
bc.sql(query)

Error found
fail: 
 org.apache.calcite.runtime.CalciteContextException: From line 9, column 18 to line 9, column 37: Column 'tpep_pickup_datetime' not found in table 'A'
Parsing Error


#### FULL OUTER JOIN
[Docs](https://docs.blazingdb.com/docs/queries#full-outer)

```sql
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
FULL OUTER JOIN table_B 
ON (table_A.column_A = table_B.column_B)
```

In [7]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi_a', '../../../data/sample_taxi.parquet')
bc.create_table('taxi_b', '../../../data/sample_taxi.csv', header=0)

BlazingContext ready


#### FULL OUTER JOIN SINGLE COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        FULL OUTER JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime
            '''
bc.sql(query)

#### FULL OUTER JOIN MULTI-COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        FULL OUTER JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime AND taxi_a.tpep_pickup_datetime = taxi_b.tpep_pickup_datetime
            '''
bc.sql(query)

#### LEFT OUTER JOIN
[Docs](https://docs.blazingdb.com/docs/queries#left-outer)

```sql
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
LEFT JOIN table_B 
ON (table_A.column_A = table_B.column_B)
```

In [6]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi_a', '../../../data/sample_taxi.parquet')
bc.create_table('taxi_b', '../../../data/sample_taxi.csv', header=0)

BlazingContext ready


#### LEFT JOIN SINGLE COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        LEFT JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime
            '''
bc.sql(query)

#### LEFT JOIN MULTI-COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        LEFT JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime AND taxi_a.tpep_pickup_datetime = taxi_b.tpep_pickup_datetime
            '''
bc.sql(query)

### ORDER BY
[Docs](https://docs.blazingdb.com/docs/queries#order-by)

```sql
SELECT column_A, column_B FROM table_A ORDER BY column_A
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

#### ORDER BY without Specificying (defaults to Ascending)

In [ ]:
bc.sql('select passenger_count, fare_amount FROM taxi ORDER BY trip_distance')

#### ORDER BY Specifying Ascending

In [ ]:
bc.sql('select passenger_count, fare_amount from taxi ORDER BY trip_distance ASC')

#### ORDER BY Specifying Descending

In [ ]:
bc.sql('select passenger_count, fare_amount from taxi ORDER BY trip_distance DESC')

### Nested Queries
[Docs](https://docs.blazingdb.com/docs/queries#nested-queries)

A query nested inside another query that uses values from the outer query may appear elsewhere. 

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

In [ ]:
query = '''
        select 
            pickup_time, passenger_count, trip_distance, cost 
        from
            (
            SELECT 
                *,
                CAST(tpep_pickup_datetime || '.0' AS timestamp) AS pickup_time, 
                total_amount - tip_amount AS cost 
            FROM 
                taxi 
                WHERE
                    passenger_count <> 1
            )
            '''
bc.sql(query)

In [ ]:
query = '''
        select 
            year(pickup_time) as pickup_year, month(pickup_time) as pickup_month, dayofmonth(pickup_time) as pickup_day,
            passenger_count, trip_distance, cost 
        from
            (
            SELECT 
                *,
                CAST(tpep_pickup_datetime || '.0' AS timestamp) AS pickup_time, 
                total_amount - tip_amount AS cost 
            FROM 
                taxi 
                WHERE
                    passenger_count <> 1
            )
            '''
bc.sql(query)

### SELECT
[Docs](https://docs.blazingdb.com/docs/queries#select)

```sql
SELECT * FROM table_a
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

In [ ]:
bc.sql('SELECT * from taxi')

In [ ]:
bc.sql('SELECT passenger_count, trip_distance, fare_amount from taxi')

### WHERE
[Docs](https://docs.blazingdb.com/docs/queries#where) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#WHERE)

Query data and filter rows with a condition(s).

```sql
SELECT * FROM table_a WHERE condition_a
```

In [8]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../../../data/sample_taxi.parquet')

BlazingContext ready


In [ ]:
bc.sql('select * from taxi WHERE passenger_count = 2')

#### WHERE -- Multiple Filters (AND)

In [ ]:
bc.sql('select * from taxi WHERE passenger_count > 2 AND trip_distance < 10')

#### WHERE -- Multiple Filters (OR)

In [ ]:
bc.sql('select * from taxi WHERE passenger_count >= 2 OR trip_distance <= 10')

#### WHERE -- Multiple Filters (AND/OR)

In [ ]:
query = ''' 
        select 
            * 
        from 
            taxi 
                WHERE 
                    (passenger_count <> 2 OR fare_amount > 100) 
                    AND trip_distance >= 10
                    '''
bc.sql(query)

# BlazingSQL Docs
**[Table of Contents](../TABLE_OF_CONTENTS.ipynb) | [Issues (GitHub)](https://github.com/BlazingDB/Welcome_to_BlazingSQL_Notebooks/issues)**